<a href="https://colab.research.google.com/github/halen48/PySysC-Colab/blob/main/PySysC_on_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup the enviorement

## SystemC (needs restart)

In [ ]:
import os
antigo_environ = os.environ
#antigo_environ

In [ ]:
# clone of PySysC
!git clone https://git.minres.com/SystemC/PySysC.git
!wget https://gist.githubusercontent.com/bagheriali2001/0736fabf7da95fb02bbe6777d53fabf7/raw/3b55bcbecc568b7a0be8fb7837987d5d364adcfe/systemc_v2.3.3_installation_script.sh
!bash ./systemc_v2.3.3_installation_script.sh


## PySysC

In [ ]:
# install wheel package
!python3 -m pip install wheel
# clone of PySysC
!git clone https://git.minres.com/SystemC/PySysC.git
with open('/content/PySysC/setup.py','r') as f:
  content = f.read()
with open('/content/PySysC/setup.py','w') as f:
  f.write(content.replace("sysc_home+'/lib64'", "sysc_home+'/lib-linux64'"))
!pip install --upgrade pip setuptools wheel
import os
os.environ['SYSTEMC_HOME'] = '/root/systemc-2.3.3/'
# install PySysC, for development PySysC use 'python3 -m pip install -e`
!pip install /content/PySysC/

## Conan (Needs restart)

In [ ]:
!pip install conan
!pip install conan_package_tools

In [ ]:
!conan profile update settings.compiler.libcxx=libstdc++11 default
!conan profile new default --detect
!conan remote add minres https://api.bintray.com/conan/minres/conan-repo --force

# PySysC-SC

In [ ]:
# get the PySysC-SC repo
!git clone --recursive https://git.minres.com/SystemC/PySysC-SC.git
# build the project libraries as shared libs
!mkdir -p PySysC-SC/build/Debug
!cd PySysC-SC/build/Debug && cmake ../../ -DCMAKE_BUILD_TYPE=Debug -DBUILD_SHARED_LIBS=ON
!cd /content/PySysC-SC/build/Debug/ && make -j vp_components

In [ ]:
import logging
from cppyy import gbl as cpp
import pysysc
###############################################################################
# setup  and load
###############################################################################
build_type='Debug'
logging.basicConfig(level=logging.DEBUG)
###############################################################################
myDir = os.path.dirname('/content/PySysC-SC/')
pysysc.load_systemc()
###############################################################################
logging.debug("Loading SC-Components lib")
pysysc.add_include_path(os.path.join(myDir, 'scc/src/sysc'))
pysysc.add_include_path(os.path.join(myDir, 'scc/src/common'))
pysysc.add_include_path(os.path.join(myDir, 'scc/third_party'))
pysysc.add_library('scc_sysc.h', os.path.join(myDir, 'build/%s/scc/src/sysc/libscc-sysc.so'%build_type))
###############################################################################
logging.debug("Loading Components lib")
pysysc.add_include_path(os.path.join(myDir, 'vp_components'))
pysysc.add_library('components.h', os.path.join(myDir, 'build/%s/vp_components/libvp_components.so'%build_type))
###############################################################################
# configure
###############################################################################
cpp.scc.init_logging(cpp.scc.log.INFO, 24, False);
cpp.sc_core.sc_report_handler.set_actions(cpp.sc_core.SC_ID_MORE_THAN_ONE_SIGNAL_DRIVER_, cpp.sc_core.SC_DO_NOTHING);

## Code Logic

In [ ]:
clkgen = cpp.ClkGen(cpp.sc_core.sc_module_name("clk_gen"))
rstgen = cpp.ResetGen(cpp.sc_core.sc_module_name("rst_gen"))
initiator = cpp.Initiator(cpp.sc_core.sc_module_name("initiator"))
memories = [cpp.Memory(cpp.sc_core.sc_module_name(name)) for name in ["mem0", "mem1", "mem2", "mem3"]]
router = cpp.Router[4](cpp.sc_core.sc_module_name("router"))
###############################################################################
# signals
###############################################################################
sig_clk = cpp.sc_core.sc_signal[cpp.sc_core.sc_time]("clk")
sig_rst = cpp.sc_core.sc_signal[cpp.sc_dt.sc_logic]("rst")
###############################################################################
# connect it
###############################################################################
clkgen.clk_o(sig_clk)
rstgen.reset_o(sig_rst)
initiator.socket.bind(router.target_socket)
initiator.clk_i(sig_clk)
initiator.reset_i(sig_rst)
router.clk_i(sig_clk)
router.reset_i(sig_rst)
for idx,m in enumerate(memories):
    router.initiator_socket.at(idx).bind(m.socket)
    m.clk_i(sig_clk)
    m.reset_i(sig_rst)

## Run

In [ ]:
tracer = cpp.scc.tracer('vcd_trace', 1, True)
cpp.sc_core.sc_start()
logging.debug("Done")